In [ ]:
#import required libraries
from azure.ml import MLClient
from azure.ml.entities import BatchEndpoint, BatchDeployment, Model, Environment, BatchOutputAction, BatchRetrySettings

In [ ]:
#enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(subscription_id, resource_group, workspace)

In [ ]:
#create a batch endpoint
endpoint = BatchEndpoint(name='my-batch-endpoint',
                         description='this is a sample batch endpoint',
                         tags={'foo': 'bar'})
                         
ml_client.batch_endpoints.create_or_update(endpoint)

In [ ]:
#create a batch deployment
model = Model(local_path='./mnist/model/')
env = Environment(conda_file='./mnist/environment/conda.yml', image='mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest')
deployment = BatchDeployment(name='non-mlflow-deployment',
                             description='this is a sample non-mlflow deployment',
                             endpoint_name='my-batch-endpoint',
                             model=model,
                             #code_configuration=CodeConfiguration(code_local_path='./mnist/code/', scoring_script='digit_identification.py'),
                             code_local_path='./mnist/code/',
                             scoring_script='digit_identification.py',
                             environment=env,
                             compute='batch-cluster',
                             instant_count=2,
                             max_concurrency_per_instance=2,
                             mini_batch_size=10,
                             output_action=BatchOutputAction.APPEND_ROW,
                             output_file_name='predictions.csv',
                             retry_settings=BatchRetrySettings(max_retries=3, timeout=30),
                             logging_level=LoggingLevel.INFO)
                             
ml_client.batch_deployments.create_or_update(deployment, set_default=True) # set this to the default deployment for the endpoint

In [ ]:
#invoke the endpoint for batch scoring job
job = ml_client.batch_endpoints.invoke(name='my-batch-endpoint',
                                        input_path={'folder': 'https://pipelinedata.blob.core.windows.net/sampledata/mnist'},
                                        # input_path=UriReference(folder='https://pipelinedata.blob.core.windows.net/sampledata/mnist') # or using UriReference class
                                        instance_count=4,
                                        mini_batch_size=20)
                                        
job_name = job.name

In [ ]:
#get the details of the job
batch_job = ml_client.jobs.get(name=job_name)
print(batch_job.status)
#stream the job logs
ml_client.jobs.stream(name=job_name)